In [3]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import nltk
import keras.utils
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow
from keras.preprocessing.text import Tokenizer
import string
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
           os.path.join(dirname, filename)

In [4]:
a_ap_17 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesApril2017.csv")
a_ap_18 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesApril2018.csv")
a_feb_17 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesFeb2017.csv")
a_feb_18 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesFeb2018.csv")
a_jan_17 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesJan2017.csv")
a_jan_18 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesJan2018.csv")
a_mr_17 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesMarch2017.csv")
a_mr_18 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesMarch2018.csv")
a_my_17 = pd.read_csv("/kaggle/input/nyt-comments/ArticlesMay2017.csv")

totaldata = pd.concat([a_ap_17,a_ap_18,a_feb_17,a_feb_18,a_jan_17,a_jan_18,a_mr_17,a_mr_18,a_my_17])

data = totaldata.headline
data = [i for i in data if i != "Unknown"]

In [5]:
def cleaning(a):
    a = (a).lower()
    a = re.sub('[%s]' % re.escape(string.punctuation), '', a)
    return a
data = [cleaning(x) for x in data]

In [6]:
token = Tokenizer()
token.fit_on_texts(data)
total_words = len(token.word_index) + 1 
Input = []
for i in data:
    token_list = token.texts_to_sequences([i])[0]
    for j in range(1, len(token_list)):
        n_gram = token_list[:j+1]
        Input.append(n_gram)
Input[:10]

[[381, 17],
 [381, 17, 5220],
 [381, 17, 5220, 511],
 [381, 17, 5220, 511, 4],
 [381, 17, 5220, 511, 4, 2],
 [381, 17, 5220, 511, 4, 2, 1573],
 [381, 17, 5220, 511, 4, 2, 1573, 139],
 [381, 17, 5220, 511, 4, 2, 1573, 139, 5],
 [381, 17, 5220, 511, 4, 2, 1573, 139, 5, 1930],
 [7, 69]]

In [7]:
max_sequence_len = len(max(Input, key=len))
Input = np.array(pad_sequences(Input, maxlen=max_sequence_len))
predictors = Input[:,:-1]  
label = Input[:,-1] 
label =keras.utils.to_categorical(label, num_classes=total_words)

In [8]:
input_len = max_sequence_len - 1
model = Sequential()
model.add(Embedding(total_words,300, input_length=input_len))
model.add(LSTM(150))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')   
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 300)           3645000   
                                                                 
 lstm (LSTM)                 (None, 150)               270600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense (Dense)               (None, 12150)             1834650   
                                                                 
Total params: 5,750,250
Trainable params: 5,750,250
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(predictors, label, epochs=70)

Epoch 1/70
1620/1620 [==============================] - 28s 14ms/step - loss: 7.8699
Epoch 2/70
1620/1620 [==============================] - 11s 7ms/step - loss: 7.2732
Epoch 3/70
1620/1620 [==============================] - 10s 6ms/step - loss: 6.7617
Epoch 4/70
1620/1620 [==============================] - 10s 6ms/step - loss: 6.1974
Epoch 5/70
1620/1620 [==============================] - 9s 6ms/step - loss: 5.5740
Epoch 6/70
1620/1620 [==============================] - 10s 6ms/step - loss: 4.9255
Epoch 7/70
1620/1620 [==============================] - 10s 6ms/step - loss: 4.2898
Epoch 8/70
1620/1620 [==============================] - 10s 6ms/step - loss: 3.7013
Epoch 9/70
1620/1620 [==============================] - 10s 6ms/step - loss: 3.1797
Epoch 10/70
1620/1620 [==============================] - 9s 6ms/step - loss: 2.7264
Epoch 11/70
1620/1620 [==============================] - 9s 6ms/step - loss: 2.3551
Epoch 12/70
1620/1620 [==============================] - 10s 6ms/step - loss

In [41]:
def generate_text(model, token, max_sequence_len, seed_text):
    next_words = 10
    for _ in range(next_words):
        token_list = token.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        
        predicted = np.argmax(predicted_probs) + 1
        
        output_word = ""
        for word,index in token.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [24]:
generated_text = generate_text(model, token, max_sequence_len, seed_text = "The crypto is going to")
print("Generated text: ",generated_text)

Generated text:  The Crypto Is Going To A Redraw To Economy On A Force Kicked In Kushner


In [26]:
generated_text = generate_text(model, token, max_sequence_len, seed_text = "Begginer")
print("Generated text: ",generated_text)

Generated text:  Begginer Win New Case It Say Just Just Crossword A Story


In [42]:
generated_text = generate_text(model, token, max_sequence_len, seed_text = "The real name of Trump")
print("Generated text: ",generated_text)

Generated text:  The Real Name Of Trump Act A Bribery Deal Three Age In A Toronto Off
